# CNN TRAINING
This notebook details the code that trains the CNNs described in [1]. These CNNs are trained on simulated gravitational lenses, up to 4 Euclid bands can be used as in put (H, Y, J, VIS). Once the CNN has been trained for the desired number of epochs, the model is updated with the weights that achieved the lowest validation loss during training. This trained CNN model is put into Eval mode and the test data is fed through the CNN. The model outputs on the test data is saved and compare to the truth table. The false positives, false negatives, true positives, and true negatives are saved. The Area Under the ROC Curve (AUC) is calculated and saved. 



The training process is saved along with any weights that generate a new minimum validation loss. The  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import glob
import torch.utils.data as data_utils
import math 
import time
import datetime
from astropy.io import fits
from sklearn import metrics
from Load_Images import J_LoadImages, Y_LoadImages, H_LoadImages, JYH_LoadImages, VIS_LoadImages, OU66_LoadImages, OU200_LoadImages
from CNN_Networks import J_CNN, Y_CNN, H_CNN, JYH_CNN, VIS_CNN, OU66_CNN, OU200_CNN

## CONSTANTS 
This section describes the constants that can be changed in this code (aside from specification of the desired CNN). 
NNNumbers is set to 0 as this is the number of new minimum validation losses created during training of the CNN.

Parameters of the CNN are defined her lr is the Learning Rate of the CNN. Threshold is the threshold for deciding what class each output is predicting. batch_size is the batch size of the CNN. This code will only accept whole batches, any batches that have less images than batch_size will be ignored. epochs is the amount of times the CNN sees the entire dataset during training. 
device specifies if the CNN will be trained on a cpu and a gpu if there is one avaliable.

The amount of images used in this notebook are specified by TrainImages, ValidationImages, and TestImages. 

Band and date alter the filename of all the files saved by this notebook to indivialise each run of this code.

In [ ]:
# Constants
NNNumbers = 0
lr = 3e-4 
Threshold = 0.5
batch_size = 500
epochs = 150
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TrainImages = 45000
ValidationImages = 3000
TestImages = 12000

Band = 'JYH'
date = 'Sept_21'

## Load Data 
There exists a CSV file that contains the Euclid ID, the truth value, and many other parameters about the image. The truth value was futher clarified so that any lens image with a n_source_im > 0, mag_eff > 1.6, and n_pix_source > 20 is defined as a lens and any other lenses are defined as a non-lens.
This function outputs pairs of Euclid IDs and their truth value.

In [ ]:
def LoadData(Number):
    
    df = pd.read_csv('EuclidDataFile2.0train.csv')
    n_source_im = df['n_source_im'].values
    mag_eff = df['mag_eff'].values
    n_pix_source = df['n_pix_source'].values
    ID = df['Euclid ID'].values
    
    Output = np.zeros((Number,2))
    
    for i in range(Number):
        if n_source_im[i] > 0:
            if mag_eff[i] > 1.6:
                if n_pix_source[i] > 20:
                Output[i,:] = [int(ID[i]), 1]
                else:
                Output[i,:] = [int(ID[i]), 0]
            else:
                Output[i,:] = [int(ID[i]), 0]
        else:
                Output[i,:] = [int(ID[i]), 0]
    return Output

## Load Images
Specify here which one of the CNNs you wish to train. This will create two outputs: Images, which is a numpy array of dimensions [1,bands, width, height] and NewOutput, a numpy array of pairs of values which are Euclid ID and the truth value. The Images width and height are 66 pixels for any CNN that uses just NISP bands, if the CNN uses the VIS band the width and height are 200 pixels.

In [ ]:
Images, NewOutput = J_LoadImages(Output)
Images, NewOutput = Y_LoadImages(Output)
Images, NewOutput = H_LoadImages(Output)
Images, NewOutput = JYH_LoadImages(Output)
Images, NewOutput = VIS_LoadImages(Output)
Images, NewOutput = OU66_LoadImages(Output)
Images, NewOutput = OU200_LoadImages(Output)

## Preparing Data For CNN

The data are moved to a GPU if one is avaliable. The Image data and the truth table are converted from numpy arrays into PyTorch tensors. The image data and the truth table are split into the training, validation, and test sets. If the size of the TensorDataset is not exactly divisible by the batch size the remainders are removed from the dataset. 

In [ ]:
Output = NewOutput

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Images = torch.from_numpy(Images).float().to(device)
Output = torch.from_numpy(Output).type(torch.LongTensor).to(device)
Output.type(torch.LongTensor) 
intiallabels = Output[:,1]
labels = intiallabels

train = data_utils.TensorDataset(Images[:TrainImages,:], intiallabels[:TrainImages])
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)

validate = data_utils.TensorDataset(Images[TrainImages:TrainImages+ValidationImages,:], intiallabels[TrainImages:TrainImages+ValidationImages])
validate_loader = data_utils.DataLoader(validate, batch_size=batch_size, shuffle=False, drop_last=True)
#validate_loader = train_loader

test = data_utils.TensorDataset(Images[TrainImages+ValidationImages:TrainImages+ValidationImages+TestImages,:], intiallabels[TrainImages+ValidationImages:TrainImages+ValidationImages+TestImages])
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)

classes = ['No Lens', 'Lens']

## Load CNN 
Here we load the CNN architectures that we want to use in this work. 


In [ ]:
model = J_CNN().to(device)
model = Y_CNN().to(device)
model = H_CNN().to(device)
model = JYH_CNN().to(device)
model = VIS_CNN().to(device)
model = OU66_CNN().to(device)
model = OU200_CNN().to(device)

## Prepare Data 
Here we determine the amount of lenses and non-lenses in the training data. We use this to calculate a weighting for the Cross Entropy Loss to balance the data.

In [ ]:
no_lens = Output[:TrainImages,1].sum()
no_nonLenses = len(Output[:TrainImages]) - no_lens
weight_ten = torch.tensor([float(no_lens)/len(Output[:TrainImages]),float(no_nonLenses)/len(Output[:TrainImages])]).to(device)
criterion = nn.CrossEntropyLoss(weight_ten)
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

## Prepare For Training 
This prepares empty lists to store data generated during training about the training process.

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

# number of epochs to train the model
n_epochs = epochs

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

timecount = []
epochall = []
valloss = []
valacc = []
trainloss = []
trainacc = []

## Training Epochs

In [ ]:
for epoch in range(n_epochs):
    # monitor training loss
    epochStartTime = time.time()#####
    correct = 0.0
    cum_loss = 0.0
    counter = 0.0

    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        if train_on_gpu:
            data = data.cuda()
            target = target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output, softmaxProbabilities = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        # Accuracy
        max_scores, max_labels = output.data.max(1)
        correct += (max_labels == target.data).sum()
        counter += data.size(0)
        train_acc = 100 * float(correct) / counter######
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    correct = 0.0
    cum_loss = 0.0
    counter = 0.0
    for data, target in validate_loader:
        if train_on_gpu:
            data = data.cuda()
            target = target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output, softmaxProbabilities = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
                # Accuracy
        max_scores, max_labels = output.data.max(1)
        correct += (max_labels == target.data).sum()
        counter += data.size(0)
        val_acc = 100 * float(correct) / counter######
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(validate_loader.dataset)
    
    epochFinishTime = time.time()
    print('Epoch: {} tTraining Loss: {:.6f} tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    print('Epoch: {} tTraining Acc: {:.6f} tValidation Acc: {:.6f}'.format(
        epoch+1, 
        train_acc,
        val_acc
        ))
    print('Time: '+ str(epochFinishTime - epochStartTime))
    
    timecount.append(epochFinishTime - epochStartTime)
    epochall.append(epoch+1)
    trainloss.append(train_loss)
    trainacc.append(train_acc)
    valloss.append(valid_loss)
    valacc.append(val_acc) 
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'Checkpoints/Pytorch1_model_'+str(date)+'_'+str(Band)+'_Explore_NN_BSGL2_'+str(NNNumbers)+'.pt')
        torch.save(optimizer.state_dict(), 'Checkpoints/PyTorch1_Optim_'+str(date)+'_'+str(Band)+'_Explore_NN_BSGL2_'+str(NNNumbers)+'.pt')
        valid_loss_min = valid_loss
        NNNumbers = NNNumbers +1


df = pd.DataFrame({"Time":timecount, "Epoch": epochall, "Valloss": valloss, "Valacc": valacc, "trainloss":trainloss, "trainacc":trainacc})
df.to_csv('Data/Explore_NN_BSGL2_'+str(date)+'_'+str(Band)+'_TrainingData.csv', index=False)

## Load Best Model Weights
The set of weights which achieved the lowest validation loss are loaded into the CNN. This set of weights are used for use on the test data.

In [ ]:
model.load_state_dict(torch.load('Checkpoints/Pytorch1_model_'+str(date)+'_'+str(Band)+'_Explore_NN_BSGL2_'+str(NNNumbers-1)+'.pt'))

In [ ]:
# initialize lists to monitor test loss and accuracy
totalPred = []
totalLabel = []
totalSoftmaxProbabilities = []

test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

## Evaluate the Model on the Test Data

In [ ]:
model.eval() # prep model for evaluation

for data, target in test_loader:
    data = data.to(device)
    target = target.type(torch.LongTensor) 
    target = target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output, softmaxProbabilities = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        totalLabel.append(label.cpu().numpy())
    totalPred.extend(pred.cpu().numpy())
    totalSoftmaxProbabilities.extend(softmaxProbabilities.detach().cpu().numpy())

In [ ]:
test_loss = test_loss/len(test_loader.dataset)

## Calculate FP, TP, FN, TN

In [ ]:
totalLabel = np.array(totalLabel)
totalPred = np.array(totalPred)
totalSoftmaxProbabilities = np.array(totalSoftmaxProbabilities)

lass = []
for i in range(len(totalLabel)):
    if totalSoftmaxProbabilities[i,0] < Threshold: # Classified as Non-Lens
        if totalLabel[i] == 0: 
            lass.append('False Positive')
        else:
            lass.append('True Positive')
    else:
        if totalLabel[i] == 1: 
            lass.append('False Negative')
        else:
            lass.append('True Negative')
lass = np.array(lass)


## Save 
Here the CNN predictions for the test set are saved. The AUC is calculated for the test set and the data to plot this curve is saved as a csv file.

In [ ]:
df = pd.DataFrame({"Euclid ID":Output[TrainImages+ValidationImages:TrainImages+ValidationImages+TestImages, 0].cpu(),"Probability of Non-Lens": totalSoftmaxProbabilities[:,0], "Probability of Lens": totalSoftmaxProbabilities[:,1], "Classification": lass,"Target":totalLabel})
df.to_csv('Data/Explore_NN_BSGL2_'+str(date)+'_'+str(Band)+'_Results.csv', index=False)


totalPred = np.array(totalPred)
totalLabel = np.array(totalLabel)


a, b, c = metrics.roc_curve(totalLabel,totalSoftmaxProbabilities[:,1]) # Probability of being a lens
roc_auc = metrics.auc(a,b)

df = pd.DataFrame({"a":a, "b":b})
df.to_csv("Data/ROC_CURVE_Explore_NN_BSGL2_"+str(date)+"_"+str(Band)+".csv", index=False)

# References

[1] Detecting gravitational lenses using machine learning: exploring interpretability and sensitivity to rare lensing configurations